<a href="https://colab.research.google.com/github/DariusTheGeek/Flood-Prediction-in-Malawi--Zindi-Competition/blob/master/Malawi_Flood_Prediction__starter_code__by_DariusMoruri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the Necessary Libraries

In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np

### numpy version '1.18.2'

In [ ]:
np.__version__

'1.18.2'

### pandas version '1.0.1'

In [ ]:
pd.__version__

'1.0.1'

### lightgbm version 2.3.1

In [ ]:
!pip install lightgbm==2.3.1
lgb.__version__

Le chemin d'accŠs sp‚cifi‚ est introuvable.


'2.3.1'

## Reading the Data

In [ ]:

sub = pd.read_csv('SampleSubmission (1).csv')


In [ ]:
df=pd.read_csv('Train.csv')

## Data Cleaning

### for this part I have used the start code: https://github.com/DariusTheGeek/Flood-Prediction-in-Malawi--Zindi-Competition to prepare the train and the test

In [ ]:
precip_features_2019 = []
precip_features_2015 = []
for col in df.columns:
    if '2019' in col:
        precip_features_2019.append(col)
    elif 'precip 2014' in col:
        precip_features_2015.append(col)
    elif 'precip 2015' in col:
        precip_features_2015.append(col)
train=tain = df[df.columns.difference(precip_features_2019)]
precip_features_2019.extend(['X',	'Y',	'elevation', 'LC_Type1_mode',	'Square_ID'])
test = df[precip_features_2019]
new_2015_cols = {}
for col, number in zip(precip_features_2015, range(1, len(precip_features_2015) + 1)):

    if 'precip' in col:
        new_2015_cols[col] = 'week_' + str(number) + '_precip'
new_2019_cols = {}
for col, number in zip(precip_features_2019, range(1, len(precip_features_2019) + 1)):
    if 'precip' in col:
        new_2019_cols[col] = 'week_' + str(number) + '_precip'
train.rename(columns = new_2015_cols, inplace = True)
test.rename(columns = new_2019_cols, inplace = True)
target = train.target_2015
train, test = train.align(test, join = 'inner', axis = 1)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
X = train.drop(['Square_ID'], axis = 1)
y = target

D:\conda\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  errors=errors,


### key point: The flood may occur in consequence of heavy precipitation that occur during a period less than 18 weeks. For that reason we have just worked on the period with a continuios heavy precip.


In [ ]:
train[train.columns[5:-1]].describe().loc[['mean']]

,week_1_precip,week_2_precip,week_3_precip,week_4_precip,week_5_precip,week_6_precip,week_7_precip,week_8_precip,week_9_precip,week_10_precip,week_11_precip,week_12_precip,week_13_precip,week_14_precip,week_15_precip,week_16_precip
mean,1.299748,1.882648,0.852641,8.367555,9.023455,9.657713,23.25205,28.164769,57.42755,1.149752,34.701142,28.358295,12.323302,3.629145,17.214365,8.887374


### Max precip within 2 weeks

In [ ]:
train['week_7_precip_']=train['week_7_precip']+train['week_6_precip']
test['week_7_precip_']=test['week_7_precip']+test['week_6_precip']
train['week_8_precip_']=train['week_8_precip']+train['week_7_precip']
test['week_8_precip_']=test['week_8_precip']+test['week_7_precip']
train['week_9_precip_']=train['week_9_precip']+train['week_8_precip']
test['week_9_precip_']=test['week_9_precip']+test['week_8_precip']
train['max_2_weeks']=train[['week_7_precip_','week_8_precip_','week_9_precip_']].apply(np.max,axis=1)
test['max_2_weeks']=test[['week_7_precip_','week_8_precip_','week_9_precip_']].apply(np.max,axis=1) 

In [ ]:

X1=train[['LC_Type1_mode', 'X', 'Y', 'elevation','week_7_precip', 'week_8_precip', 'week_9_precip','max_2_weeks']]
sub1=test[['LC_Type1_mode', 'X', 'Y', 'elevation','week_7_precip', 'week_8_precip', 'week_9_precip','max_2_weeks']]
X1.columns=sub1.columns


### Index of the week within the max of precip

In [ ]:
def index(col):
    l=list(col)
    return l.index(max(l))

X1['max_index']=train[['week_6_precip', 'week_7_precip', 'week_8_precip','week_9_precip']].apply(index,axis=1)
sub1['max_index']=test[['week_6_precip', 'week_7_precip', 'week_8_precip','week_9_precip']].apply(index,axis=1)



D:\conda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  """

D:\conda\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  


### Precip variation over different weeks

In [ ]:
sub1['slope_8_7']=((test['week_8_precip']/test['week_7_precip'])>1)*1
X1['slope_8_7']=((train['week_8_precip']/train['week_7_precip'])>1)*1


sub1['slope_9_8']=((test['week_9_precip']/test['week_8_precip'])>1)*1
X1['slope_9_8']=((train['week_9_precip']/train['week_8_precip'])>1)*1


D:\conda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  """Entry point for launching an IPython kernel.

D:\conda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  

D:\conda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 

A value is trying to be set on a copy of a slice from a DataFrame.

Try using .loc[row_indexer,col_indexer] = value instead



See the caveats in the documentation: https://pand

### outlier detection

In [ ]:
from sklearn.covariance import EllipticEnvelope
clf2 = EllipticEnvelope(contamination=.17,random_state=0)
clf2.fit(X1)
ee_scores = pd.Series(clf2.decision_function(X1))
clusters2 = clf2.predict(X1)
X1['target']=target
X1['pred']=clusters2
X1=X1[X1['pred']!=-1]
X1,y=X1.drop(columns=['pred','target']),X1['target']

D:\conda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-62.156597799790795 > -63.517750296664858). You may want to try with a higher value of support_fraction (current value: 0.503).

  RuntimeWarning)

D:\conda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-60.551761565457447 > -61.007147944574804). You may want to try with a higher value of support_fraction (current value: 0.503).

  RuntimeWarning)

D:\conda\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-61.346057144722828 > -61.944318737644792). You may want to try with a higher value of support_fraction (current value: 0.501).

  RuntimeWarning)

D:\conda\lib\site-packages\ipykernel_launcher.

### using Soil organic carbon stock: (external data):
### extrcted from "https://soilgrids.org/"


In [ ]:
s=pd.read_csv('ss.csv')
s.drop(columns='_soilvarie',axis=1,inplace=True)
X1=X1.merge(s,on=['X','Y'],how='left')
sub1=sub1.merge(s,on=['X','Y'],how='left')

In [ ]:
def metric(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

### Training model

In [ ]:
params = { 'learning_rate':0.07,'max_depth':8}
X=X1
X_test=sub1

n_estimators = 221

n_iters = 5
preds_buf = []
err_buf = []
for i in range(n_iters): 
    x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=i)
    d_train = lgb.Dataset(x_train, label=y_train)
    d_valid = lgb.Dataset(x_valid, label=y_valid)
    watchlist = [d_valid]

    model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

    preds = model.predict(x_valid)
   
 
    err_buf.append(metric(model.predict(x_valid),y_valid))
    
    
    preds = model.predict(X_test)
    
    preds_buf.append(preds)

print('Mean RMSLE = ' + str(np.mean(err_buf)) + ' +/- ' + str(np.std(err_buf)))
# Average predictions
preds1 = np.mean(preds_buf, axis=0)

Mean RMSLE = 0.06602787044143263 +/- 0.003301317857027148


In [ ]:
def check(col):
    if col<0:
        return 0
    elif col>1:
        return 1
    else:
        return col
preds=preds1
preds-=0.08
submission_df = pd.DataFrame({'Square_ID': test.Square_ID, 'target_2019': preds}) 
submission_df['target_2019']=submission_df['target_2019'].apply(check)
submission_df.to_csv('lili.csv', index = False)